In [1]:
import pandas as pd
import numpy as np
import datetime as dt

# Şimdilik, veriyi direkt olarak 'data' klasöründen okuyormuş gibi varsayalım.
try:
    df = pd.read_csv("C:/Users/safye/Desktop/Customer_Churn_CLV_Project/Customer_Churn_CLV_Project/data/OnlineRetail.csv/OnlineRetail.csv", encoding = 'unicode_escape') 
    print("Veri başarıyla yüklendi.")
    df_copy = df.copy() 
except FileNotFoundError:
    print("HATA: Lütfen dosyanın 'data/Online_Retail.csv' yolunda olduğundan emin olun.")

# İlk kontrolleri tekrar yapalım:
print("\n--- Eksik Değer Kontrolü (isnull) ---")
print(df.isnull().sum())

Veri başarıyla yüklendi.

--- Eksik Değer Kontrolü (isnull) ---
InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64


In [ ]:
# Temizlik ve Dönüşümler

# 1. Eksik Müşteri ID'lerini Silme
# Toplam 135,080 satırı siliyoruz.
print(f"Başlangıç Müşteri ID NaN Sayısı: {df['CustomerID'].isnull().sum()}")
df.dropna(subset=['CustomerID'], inplace=True) 

Başlangıç Müşteri ID NaN Sayısı: 0


Neden İptal Edilen İşlemleri Çıkarıyoruz

1= Monetary (M) İçin: 
Müşteri 100 TL'lik bir alışveriş yaptı, sonra bu siparişi iptal etti. Eğer iptal edilen bu işlemi veri setinde bırakırsak, müşterinin toplam harcaması (M) hatalı olarak 100 TL daha fazla görünür. Modelimiz, aslında parayı geri almış bir müşteriyi yüksek değerli sanır.

2= Frequency (F) İçin:
 Aynı şekilde, iptal edilen bir sipariş, müşterinin toplam sipariş sayısını (F) hatalı olarak artırır. Model, aslında vazgeçmiş bir müşteriyi sadık sanabilir.Bu yüzden bu satırları çıkararak sadece tamamlanmış ve ödenmiş gerçek işlemleri saklıyoruz.

In [6]:
# 2. İptal Edilen İşlemleri Çıkarma
df = df[~df["InvoiceNo"].astype(str).str.contains("C", na=False)] 

# 3. Harcama Miktarı Sıfır ve Negatif Olanları Çıkarma
# Hata olarak girilmiş veya anlamsız (sıfır, negatif) miktarları ve fiyatları veri setinden temizleriz.
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]

# 4. Veri Tipi Dönüşümleri
df["CustomerID"] = df["CustomerID"].astype(int) 

# 5. Monetary (Harcama) Değişkenini Oluşturma (M)
# Müşteri Harcaması = Miktar * Birim Fiyat
df["TotalPrice"] = df["Quantity"] * df["UnitPrice"]

# 6. InvoiceDate (Tarih) Tipini Dönüştürme
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])

print("--- Temizlik Başarıyla Tamamlandı ---")
print(f"Toplam Kalan Satır Sayısı: {len(df)}")
print(f"Toplam Tekil Müşteri Sayısı: {df['CustomerID'].nunique()}")

--- Temizlik Başarıyla Tamamlandı ---
Toplam Kalan Satır Sayısı: 397884
Toplam Tekil Müşteri Sayısı: 4338


!!!
Bizim projemizin amacı tahmin yapmaktır: Müşteri A, gelecekte (önümüzdeki 30 gün içinde) bizi terk edecek mi? sorusunu soruyoruz bunun için bir referans zamanı belirlemeliyiz bu referans zamanı da veri setimizin hepsinin en son işlme yaptığı günden bir sonraki gündür. 

In [ ]:
import datetime as dt 

# 1. Veri Setindeki En Son İşlem Tarihini Bulma
last_date = df["InvoiceDate"].max()

# 2. Referans Tarihini (Bugün'ü Temsil Eden Tarih) Belirleme
ANALYSIS_DATE = last_date + dt.timedelta(days=1) 
print(f"Analiz Referans Tarihi (Bugün): {ANALYSIS_DATE}")

Analiz Referans Tarihi (Bugün): 2011-12-10 12:50:00


In [ ]:
rfm_df = df.groupby('CustomerID').agg({
    # Recency (R): Müşterinin son alışverişinden kaç gün geçtiğini buluruz.Gün olarak.
    "InvoiceDate": lambda InvoiceDate: (ANALYSIS_DATE - InvoiceDate.max()).days,
    
    # Frequency (F): Tekil sipariş sayısı.
    # Müşterinin toplam kaç farklı sipariş verdiğini buluyoruz.
    "InvoiceNo": lambda InvoiceNo: InvoiceNo.nunique(),
    
    # Monetary (M): Toplam harcama. Müşterinin bize kazandırdığı toplam parayı buluruz.
    "TotalPrice": lambda TotalPrice: TotalPrice.sum()
})

rfm_df.columns = ['Recency', 'Frequency', 'Monetary']

print("\n--- RFM Tablosunun İlk 5 Satırı (X Değişkenleri Çekirdeği) ---")
print(rfm_df.head())


--- RFM Tablosunun İlk 5 Satırı (X Değişkenleri Çekirdeği) ---
            Recency  Frequency  Monetary
CustomerID                              
12346           326          1  77183.60
12347             2          7   4310.00
12348            75          4   1797.24
12349            19          1   1757.55
12350           310          1    334.40


Proje tanımımıza göre, müşterinin önümüzdeki 30 gün içinde geri gelip gelmeyeceğini tahmin etmeliyiz.

Geçmiş (X): ANALYSIS_DATE'e kadar olan her şey.

Gelecek (Y): ANALYSIS_DATE'ten sonraki 30 gün.

Bu kod bloğu, müşterinin geçmişteki davranışlarına (RFM/X) bakarak tahmin edeceğimiz gelecekteki durumunu (CHURN/Y) tanımlar ve hesaplar.

In [ ]:
# Hücre 5: Kayıp (Y) Değişkenini Oluşturma

# Tahmin yapacağı zaman aralığını belirtiyoruz
# Başlangıç: ANALYSIS_DATE (Örn: 1 Ocak 2012)
target_start_date = ANALYSIS_DATE 
# Bitiş: ANALYSIS_DATE'ten 30 gün sonrası
target_end_date = ANALYSIS_DATE + dt.timedelta(days=30) 

print(f"RFM (X) hesaplandı: {ANALYSIS_DATE} tarihine kadar.")
print(f"Tahmin Penceresi (Y): {target_start_date} ile {target_end_date} arası (30 Gün)")

RFM (X) hesaplandı: 2011-12-10 12:50:00 tarihine kadar.
Tahmin Penceresi (Y): 2011-12-10 12:50:00 ile 2012-01-09 12:50:00 arası (30 Gün)


In [ ]:
# 2. 30 Gün İçinde Geri Gelen Müşterileri Bulma (Non-Churn Grubu)

# Sadece hedef zamandaki işlemleri filtrele
target_df = df[(df["InvoiceDate"] >= target_start_date) & (df["InvoiceDate"] < target_end_date)]

# Bu dönemde alışveriş yapan tekil müşteri ID'lerini (Y=0 olanları) listele
loyal_customers = target_df["CustomerID"].unique()

# 3. Y Değişkenini (CHURN) rfm_df'e Ekleme
# Adım A: Varsayılan olarak herkesi Kayıp (1) kabul et
rfm_df["CHURN"] = 1

# Adım B: Hedef dönemde alışveriş yapanları (loyal_customers) Devam (0) olarak güncelle
rfm_df.loc[rfm_df.index.isin(loyal_customers), "CHURN"] = 0

print("\n--- Churn Durumu Dağılımı ---")
# 1 ve 0 sayısını kontrol etme. Bu, modelimizin sınıf dağılımını gösterir.
print(rfm_df["CHURN"].value_counts()) 


--- Churn Durumu Dağılımı ---
CHURN
1    4338
Name: count, dtype: int64


Sınıf Dengesizliği sorunu oluştu:

target_start_date = ANALYSIS_DATE
ANALYSIS_DATE'in kendisi (2011-12-10 12:50:00), veri setindeki son işlemin hemen sonrası olduğu için, ondan sonraki 30 gün içinde kaydedilmiş işlem verisi yok (veya çok az).

target_df = df[...]
Veri seti orada bittiği için, bu filtreleme sonucunda target_df (Gelecekteki İşlemler) tablosu boş çıktı.

loyal_customers = target_df["CustomerID"].unique()
Boş çıkan target_df'ten dolayı, loyal_customers (Devam Eden Müşteriler) listesi de boş oldu.

rfm_df.loc[... , "CHURN"] = 0
Hiç loyal_customers olmadığı için, varsayılan $Y=1$ etiketinden $Y=0$ olarak değiştirilen hiç müşteri olmadı.

In [13]:
# Hücre 6: Final Kontrolü

# RFM değerleri ile CHURN etiketini birleştirmiş final tablomuz
final_df = rfm_df[['Recency', 'Frequency', 'Monetary', 'CHURN']]

print("--- Final Model Veri Seti (X ve Y) ---")
print(final_df.head())

# Kayıp (1) olan müşterilerin Kayıp Olmayan (0) müşterilere oranını kontrol edelim
churn_rate = final_df['CHURN'].value_counts(normalize=True).mul(100).round(2)
print("\n--- Kayıp Oranı (Sınıf Dengesi) ---")
print(churn_rate)

--- Final Model Veri Seti (X ve Y) ---
            Recency  Frequency  Monetary  CHURN
CustomerID                                     
12346           326          1  77183.60      1
12347             2          7   4310.00      1
12348            75          4   1797.24      1
12349            19          1   1757.55      1
12350           310          1    334.40      1

--- Kayıp Oranı (Sınıf Dengesi) ---
CHURN
1    100.0
Name: proportion, dtype: float64


Düzeltme

In [18]:
# Hücre 7: Yeni Simülasyon Noktasını Belirleme

import datetime as dt # Tarih işlemleri için kütüphane

# 1. Veri setinin en son tarihini bulma
simulated_max_date = df["InvoiceDate"].max()

# 2. YENİ Referans Tarihi (Simülasyon Bugünü): En son tarihten 90 gün öncesi
NEW_ANALYSIS_DATE = simulated_max_date - dt.timedelta(days=90)

print(f"Orijinal Son İşlem Tarihi: {simulated_max_date}")
print(f"Yeni Simülasyon Analiz Tarihi (X Hesaplama Sonu): {NEW_ANALYSIS_DATE}")

Orijinal Son İşlem Tarihi: 2011-12-09 12:50:00
Yeni Simülasyon Analiz Tarihi (X Hesaplama Sonu): 2011-09-10 12:50:00


In [20]:
# Hücre 8: Yeni RFM Hesaplama (df_train ile)

# 1. Eğitim verisi setini (RFM'in hesaplanacağı kısım) filtreleme
df_train = df[df["InvoiceDate"] <= NEW_ANALYSIS_DATE]

# 2. Yeni RFM Hesaplama (df_train'i kullanarak)
rfm_df = df_train.groupby('CustomerID').agg({
    "InvoiceDate": lambda x: (NEW_ANALYSIS_DATE - x.max()).days, # Recency (R)
    "InvoiceNo": lambda x: x.nunique(), # Frequency (F)
    "TotalPrice": lambda x: x.sum() # Monetary (M)
})
rfm_df.columns = ['Recency', 'Frequency', 'Monetary']

In [21]:
# Hücre 9: Y (CHURN) Değişkenini Yeniden Tanımlama ve Kontrol

# 1. Tahmin Penceresi: NEW_ANALYSIS_DATE + 30 Gün
target_start_date = NEW_ANALYSIS_DATE
target_end_date = NEW_ANALYSIS_DATE + dt.timedelta(days=30) 

# 2. Hedef pencerede alışveriş yapan müşterileri bulma (Y=0)
target_df = df[(df["InvoiceDate"] > target_start_date) & (df["InvoiceDate"] <= target_end_date)]
loyal_customers = target_df["CustomerID"].unique()

# 3. CHURN etiketini rfm_df'e ekleme
rfm_df["CHURN"] = 1
rfm_df.loc[rfm_df.index.isin(loyal_customers), "CHURN"] = 0

print("--- Yeni Churn Durumu Dağılımı ---")
print(rfm_df["CHURN"].value_counts())
print("\n--- Yeni Kayıp Oranı (Sınıf Dengesi) ---")
print(rfm_df['CHURN'].value_counts(normalize=True).mul(100).round(2))

--- Yeni Churn Durumu Dağılımı ---
CHURN
1    2414
0     956
Name: count, dtype: int64

--- Yeni Kayıp Oranı (Sınıf Dengesi) ---
CHURN
1    71.63
0    28.37
Name: proportion, dtype: float64
